In [4]:
import subprocess
import json
import re

SYSTEM_PROMPT = """

شما یک استخراج‌کننده‌ی ساختاریافته‌ی فارسی هستید. ورودی یک جمله‌ی فارسی کوتاه است که کاربر دنبال «چه چیزی» و «کجا» (و گاهی شماره تلفن) می‌گردد. وظیفه‌تان این است که **تنها** و **فقط** یک شیء JSON معتبر برگردانید با سه کلید:

{
  "searchTerm": string,   // عبارتی کوتاه که نوع کسب‌وکار/محصول را نشان می‌دهد (مثلاً "برنج فروشی" یا "رستوران")
  "address": string,      // محل/شهر/محله در قالبی کوتاه (مثلاً "فردیس کرج" یا "تهران منطقه 3"). اگر مکان مشخصی نیست، رشته‌ی خالی "".
  "phone": string         // شماره تماس اگر در جمله موجود است؛ در غیر این صورت "". تنها ارقام و علائم + یا - یا فاصله مجازند (مثلاً "+98912..." یا "02112345678").
}

قواعد سخت:
1. فقط JSON بالا را چاپ کن — هیچ متن توضیحی، پیش‌زمینه یا چیز دیگری مجاز نیست.
2. اگر نمی‌توانی مقدار درست تشخیص دهی، مقدار همان فیلد را به صورت رشته‌ی خالی "" قرار بده.
3. کوتاه و دقیق نگهدار: searchTerm را به شکل عمومی (بدون صفات اضافی مانند "بهترین") بده: مثلا "برنج فروشی" نه "بهترین برنج فروشی".
4. اگر چند مؤلفهٔ آدرس دیده شد (شهر و محله)، آنها را با فاصله جدا کن: "فردیس کرج".
5. اگر شماره تلفن یافت شد، آن را پاک‌سازی کن تا فقط ارقام و در صورت لزوم پیش‌شماره +98 باقی بماند (مثال‌های زیر را ببین).

حالا چند مثال ورودی→خروجی (فقط JSON):
Input: "بهترین برنج فروشی های فردیس کرج"
Output: {"searchTerm":"برنج فروشی","address":"فردیس کرج","phone":""}

Input: "رستوران های ارزان تهران منطقه 3 شماره تماس 021-12345678"
Output: {"searchTerm":"رستوران","address":"تهران منطقه 3","phone":"02112345678"}

Input: "قنادی شایان نزدیک میدان ولیعصر تلفن 0912 123 4567"
Output: {"searchTerm":"قنادی","address":"میدان ولیعصر","phone":"09121234567"}

Input: "نزدیک مترو، تعمیر موبایل"
Output: {"searchTerm":"تعمیر موبایل","address":"", "phone":""}


"""

def build_prompt(user_input: str) -> str:
    # prompt نهایی شامل instruction و ورودی کاربر
    return SYSTEM_PROMPT + "\n\nInput: " + user_input + "\nOutput:"

def call_ollama_cli(model_name: str, prompt: str) -> str:
    # از ollama CLI استفاده می‌کنیم؛ return متن خروجی مدل (stdout)
    proc = subprocess.run(
        ["ollama", "run", model_name, prompt],
        capture_output=True, text=True, timeout=20
    )
    if proc.returncode != 0:
        raise RuntimeError("Ollama CLI error: " + proc.stderr)
    return proc.stdout.strip()

# regex برای استخراج شماره تلفن در صورت نیاز (ایران)
PHONE_RE = re.compile(r'(\+98|0098|0)?\s*9\d{2}\s*[\- ]?\s*\d{3}\s*[\- ]?\s*\d{4}|0\d{2,3}[\- ]?\d{6,8}')

def clean_phone(raw: str) -> str:
    if not raw:
        return ""
    digits = re.sub(r'[^\d+]', '', raw)
    # تبدیل +98... به +98... و حذف صفر اول اگر لازم است
    digits = digits.replace("0098", "+98")
    # اگر با 0 شروع کند و بعد 9... است، نگه دار (مثلاً 0912...)
    return digits

def parse_or_fallback(text: str, original_input: str):
    # تلاش برای تبدیل متن مدل به JSON
    try:
        return json.loads(text)
    except Exception:
        # fallback ساده: تلاش استخراج با regex
        # searchTerm <- حذف کلمات رایج مثل "بهترین"، "کدام" و خروجی بقیه را نگه دار
        addr = ""
        phone = ""
        m = PHONE_RE.search(original_input)
        if m:
            phone = clean_phone(m.group(0))
        # تلاش برای استخراج address با پیداکردن اسامی شهر/محله (ساده: پیدا کردن کلمات بعد از "در" یا "های")
        addr_match = re.search(r'در\s+([^\d,،]+)', original_input)
        if addr_match:
            addr = addr_match.group(1).strip()
        # searchTerm: حذف واژه‌های کلی
        st = re.sub(r'\b(بهترین|کدام|کدوم|نزدیک|نزدیکترین|در)\b', '', original_input)
        # حذف اسامی محل
        if addr:
            st = st.replace(addr, '')
        # پاکسازی اضافات
        st = re.sub(r'[^آ-یa-zA-Z0-9\s]', ' ', st)
        st = ' '.join(st.split()).strip()
        # گرفتن چند کلمه اول به عنوان نوع (heuristic)
        if len(st.split()) > 3:
            st = ' '.join(st.split()[:2])
        if not st:
            st = ""
        return {"searchTerm": st, "address": addr, "phone": phone}

def extract_structured(model_name: str, user_input: str):
    prompt = build_prompt(user_input)
    try:
        raw = call_ollama_cli(model_name, prompt)
    except Exception as e:
        # در صورت خطا از fallback استفاده می‌کنیم
        return parse_or_fallback("", user_input)

    # پاکسازی خروجی و parse JSON
    raw = raw.strip()
    # ممکن است مدل گاهی متن اضافی قبل/بعد از JSON بدهد؛ تلاش برای یافتن اولین { ... }
    json_text = None
    m = re.search(r'(\{.*\})', raw, flags=re.DOTALL)
    if m:
        json_text = m.group(1)
    else:
        json_text = raw

    try:
        obj = json.loads(json_text)
        # تضمین ساختار کلیدها
        return {
            "searchTerm": obj.get("searchTerm","") if isinstance(obj.get("searchTerm",""), str) else "",
            "address": obj.get("address","") if isinstance(obj.get("address",""), str) else "",
            "phone": obj.get("phone","") if isinstance(obj.get("phone",""), str) else ""
        }
    except Exception:
        return parse_or_fallback(raw, user_input)


In [8]:
model = "partai/dorna-llama3:latest"  # اسم مدل در Ollama که با این system prompt کار می‌کند
q = "بهترین تامین کننده های گوشت و مرغ پردیس تهران"
print(extract_structured(model, q))

{'searchTerm': 'تامین کننده', 'address': '', 'phone': ''}
